In [1]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import itertools
from functools import partial

%matplotlib inline

print('cuda:', torch.cuda.is_available())

cuda: True


The code has been referenced from the follwing post :-
http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

### data loader

In [12]:
transforms = []
transforms += [torchvision.transforms.Resize(self.input_w, self.input_h)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=20)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)

NameError: name 'self' is not defined

In [3]:
transforms = []
transforms += [torchvision.transforms.Resize(self.input_w, self.input_h)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
evalset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)

In [4]:
transforms = []
transforms += [torchvision.transforms.Resize(self.input_w, self.input_h)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)

### fine-tune model

In [5]:
model = torchvision.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Sequential(torch.nn.Dropout(p=0.0),torch.nn.Linear(2048, 12))

if torch.cuda.is_available():
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()
params = list(filter(lambda p: p.requires_grad, model.parameters()))

In [10]:
def score(model, dataloader):
    # initital values
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.
    for data in dataloader: 
        # next batch
        inputs, labels = data
        print(inputs.shape, labels.shape)
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
        # model scores
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        # sum the loss and accuracy
        calc_loss += loss.data[0]
        calc_correct += torch.sum(preds == labels.data)
        calc_count += outputs.data.shape[0]       
    return round(calc_loss/calc_count, 4), round(calc_correct/calc_count, 4)

In [7]:
def train(trainloader, evalloader, validloader, epochs):
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.
    for epoch in range(epochs):
        # read data
        for data in trainloader: 
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            # calculate loss and output
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            # update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            calc_loss += loss.data[0]
            calc_correct += torch.sum(preds == labels.data)
            calc_count += outputs.data.shape[0]
        # evaluate
        print('running    loss:', (round(calc_loss/calc_count, 4), round(calc_correct/calc_count, 4)))
        print('training   loss:', score(model, evalloader))
        print('validation loss:', score(model, validloader))
        print('----------------')
    return model

### execution

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=64, shuffle=False, num_workers=4)
validloader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=False, num_workers=4)

In [11]:
# model performance with random weights in fc layer
print('baselines:')
print('training   loss:', score(model, evalloader))
print('validation loss:', score(model, validloader))

baselines:


RuntimeError: Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 109, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 109, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 91, in default_collate
    return torch.stack(batch, 0, out=out)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/functional.py", line 66, in stack
    return torch.cat(inputs, dim, out=out)
RuntimeError: inconsist

In [ ]:
# one epoch with small learning rate to ensure convergence
optimizer= torch.optim.Adam(params, lr=1e-5, weight_decay=0.)
model = train(trainloader,evalloader, validloader, 1)

In [ ]:
# increase learning rates
optimizer= torch.optim.Adam(params, lr=1e-3, weight_decay=0.)
model = train(trainloader, evalloader, validloader, 3)